### Se importan las librerías necesarias

In [1]:
import pandas as pd
import numpy as np

### Se definen funciones importantes

In [2]:
# Esta función obtiene el índice de refracción del medio y el error asociado


def indice_de_refaccion(dataframe):
    dataframe["seno_incidente"] = round(np.sin(dataframe["incidente"] * np.pi / 180), 3)
    dataframe["seno_refractado"] = round(np.sin(dataframe["refractado"] * np.pi / 180), 3)
    dataframe["indice_de_refraccion"] = round(dataframe["seno_incidente"] / dataframe["seno_refractado"], 3)
    indice = round(dataframe["indice_de_refraccion"].mean(), 3)
    desv_est = round(dataframe["indice_de_refraccion"].std(), 3)
    print(f"El indice de refracción es -> {indice} +/- {desv_est}")
    print(f'\tEl error porcentual es -> {round(desv_est / indice * 100, 3)}%')

### Se cargan los datos resultantes de las mediciones en Tracker

In [6]:
angulos_31 = pd.read_csv("../data/angulos_31.csv")
angulos_61 = pd.read_csv("../data/angulos_61.csv")
angulos_barra_gruesa = pd.read_csv("../data/angulos_barra_gruesa.csv")

### Se obtienen los índices de refracción para los diferentes medios

$$n=n_i\frac{\sin{\theta_i}}{\sin{\theta_t}}$$

donde $n_i\approx1$ es el indice de refracción del aire

##### Medio con concentración grenetina/agua 3:1

In [4]:
indice_de_refaccion(angulos_31)

El indice de refracción es -> 1.215 +/- 0.23
	El error porcentual es -> 18.93%


##### Medio con concentración grenetina/agua 1:1

In [5]:
indice_de_refaccion(angulos_61)

El indice de refracción es -> 1.334 +/- 0.121
	El error porcentual es -> 9.07%


##### Barra de silicón gruesa

In [7]:
indice_de_refaccion(angulos_barra_gruesa)

El indice de refracción es -> 1.669 +/- 0.164
	El error porcentual es -> 9.826%
